In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import numpy as np
import cv2

def escalar_imagen(image, max_width):
    original_height, original_width = image.shape
    ratio = max_width / original_width
    height = int(original_height * ratio)
    return cv2.resize(image, (max_width, height))

def mostrar_imagen(image, label, max_width):
    imagen_resized = escalar_imagen(image, max_width)
    imagen_tk = ImageTk.PhotoImage(image=Image.fromarray(imagen_resized))
    label.config(image=imagen_tk)
    label.image = imagen_tk

def butterworth_lowpass_filter(image, cutoff, order):
    rows, cols = image.shape
    crow, ccol = rows // 2, cols // 2

    # Crear filtro Butterworth
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)

    # Actualizar máscara para el filtro pasa bajos Butterworth
    mask = np.zeros((rows, cols))
    x, y = np.ogrid[:rows, :cols]
    mask_area = (x - crow) ** 2 + (y - ccol) ** 2 <= cutoff ** 2
    mask[mask_area] = 1

    # Aplicar la máscara al espectro de frecuencia y obtener la imagen filtrada
    fshift = fshift * mask
    img_back = np.fft.ifft2(np.fft.ifftshift(fshift)).real

    # Crear espectro del filtro en el dominio de la frecuencia
    filter_spectrum = mask * 255

    return img_back, filter_spectrum

def high_boost_filter(image, cutoff, order, gain):
    # Aplicar filtro pasa bajos Butterworth
    img_lowpass, _ = butterworth_lowpass_filter(image, cutoff, order)
    
    # Calcular la imagen de realce (high boost)
    img_high_boost = (gain - 1) * image + img_lowpass

    return img_high_boost

def actualizar_filtro(valor):
    global img
    n = int(slider_n.get())

    # Aplicar filtro pasa bajos Butterworth a la imagen
    img_lowpass, filtro_espectro = butterworth_lowpass_filter(img, n, 2)

    # Obtener el valor actual del slider de ganancia
    gain = slider_gain.get()

    # Aplicar filtro High Boost
    img_high_boost = high_boost_filter(img, n, 2, gain)

    # Mostrar imágenes
    mostrar_imagen(img, label_imagen, 300)
    mostrar_imagen(filtro_espectro, label_matriz, 300)
    mostrar_imagen(img_high_boost, label_g_real, 300)

image_path = "pelo_microscopio.jpg"  # Ruta de la imagen a mostrar
img = cv2.imread(image_path, 0)

ventana = tk.Tk()
ventana.title("High Boost con Filtro Pasa Bajos Butterworth")

label_imagen = tk.Label(ventana)
mostrar_imagen(img, label_imagen, 300)
label_imagen.grid(row=0, column=0, padx=10, pady=10)

label_matriz = tk.Label(ventana)
label_matriz.grid(row=0, column=1, padx=10, pady=10)

label_g_real = tk.Label(ventana)
label_g_real.grid(row=0, column=2, padx=10, pady=10)

slider_n = tk.Scale(ventana, from_=1, to=100, orient=tk.HORIZONTAL,
                    label="Tamaño del Filtro", command=actualizar_filtro)
slider_n.set(30)
slider_n.grid(row=1, column=0, columnspan=3, pady=10)

slider_gain = tk.Scale(ventana, from_=1, to=5, resolution=0.1, orient=tk.HORIZONTAL,
                       label="Ganancia (High Boost)", command=actualizar_filtro)
slider_gain.set(2)
slider_gain.grid(row=2, column=0, columnspan=3, pady=10)

ventana.mainloop()
